####Loading the Data from Kaggle

In [5]:
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tobiasbueck/multilingual-customer-support-tickets")

print("Path to dataset files:", path)

100%|██████████| 16.1M/16.1M [00:00<00:00, 133MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/tobiasbueck/multilingual-customer-support-tickets/versions/14


In [2]:
import os

path_to_dataset = "/root/.cache/kagglehub/datasets/tobiasbueck/multilingual-customer-support-tickets/versions/14"

os.listdir(path_to_dataset)

['dataset-tickets-german_normalized.csv',
 'dataset-tickets-german_normalized_50_5_2.csv',
 'aa_dataset-tickets-multi-lang-5-2-50-version.csv',
 'dataset-tickets-multi-lang-4-20k.csv',
 'dataset-tickets-multi-lang3-4k.csv']

In [13]:
BASE_DIR = "/root/.cache/kagglehub/datasets/tobiasbueck/multilingual-customer-support-tickets/versions/14/dataset-tickets-multi-lang-4-20k.csv"

df = pd.read_csv(BASE_DIR)

print(df.head(20))

                                              subject  ...             tag_8
0   Unvorhergesehener Absturz der Datenanalyse-Pla...  ...               NaN
1                            Customer Support Inquiry  ...               NaN
2                       Data Analytics for Investment  ...               NaN
3                  Krankenhaus-Dienstleistung-Problem  ...               NaN
4                                            Security  ...               NaN
5   Concerns About Securing Medical Data on 2-in-1...  ...               NaN
6   Ratung für Sicherung medizinischer Daten in Hu...  ...               NaN
7                            Problem with Integration  ...               NaN
8                                  Assistance Request  ...               NaN
9                                     Support Request  ...               NaN
10  Issue with Data Analytics Platform - Insuffici...  ...               NaN
11  Reported Issue with Project Sync Resulting in ...  ...               NaN